# Right on target

You are inside a two-dimensional box, not unlike a billard table. Somewhere else inside the box is a target you would like to hit, by making good use of the surrounding walls.

It so happens that your shots can only ever travel a fixed distance, as defined by the Laws of Physics TM. The question then becomes - in how many different direction can you fire the shots in order to hit the target? That is, how many different angles can you choose from, of course ensuring that you don't accidentially shoot yourself first! 

You'll want to allow for the following assumptions:

- the ball(s) you shoot are pointlike, as are you (and the target)
- perfect reflection, i.e. no absorption until you've reached the maximum distance
- your position != target and both are strictly within the box
- the problem is strictly two-dimensional

Inputs: The dimensions of the room (two positive integers), yours and the target's positions (they will never overlap, or be sitting on one of the walls) and the maximum travel distance.

# Solution

The most important insight is that one can generate all the paths from start to target by literally "thinking outside the box". Rather than trying to draw all the possible paths linking start and target, we can use facts from elementary geometry. In the easiest case, consider an example where the ball is reflected on exactly one wall, as shown below:


As should be clear from the sketch, we can infer the direction and distance for a shot bouncing back from the bottom wall simply by mirroring the target on the line representing the wall, then drawing a vector from the start to the mirrored target, which will have the exact same length as the actual path the ball would travel.

Of course this is fairly straigthforward when considering only one reflection, but what happens when we take into account multiple bounces? Here, the idea is to repeat what's done for the simplest case over and over again by creating a kind of grid around the original box - boxes of the same size added right, left, top and bottom, as seen here:

The point in the bottom-right box, for example, would then correspond to a shot that first hits the bottom wall, then the right one (or the other way around - you can convince yourself that often there will be multiple ways to reach any given box, but only one will be realized, depending on the position of start & target and the size of the original box). 

We furthermore notice that the x- and y-coordinates of the mirrored points follow a certain patterns. Let's introduce a few variables we'll be using throughout:

- `max_dist`: the maximum distance the ball can travel
- `target =  [tx,ty]`: target we want to hit
- `dim = [x,y]`: dimensions of the room

The first point to the left of the original box, for example, will have x-coordinate `tx +2*(x-tx) = 2x-tx`, and the one to the left will be found at `-tx`. Extending the patterns, we'll find that the mirrored coordinates are all of the form `\pm x \pm 2*n*tx` for the x-coordinate and `\pm y \pm 2*m*ty` for the y-coordinate, where m and n are intergers >=0.

A first step is therefore to create a grid of the appropriate size with all those mirrored points. What's appropriate? Well, a good measure is the maximum distance divided by the shortest side of the box, rounded to the nearest integer. Here's an image that conveys the basic idea:

Any grid bigger than this will only contain points for which the ball would run out of energy before it can hit the target. 

Now, we don't really care for the mirrored points as such, but for how far away they are from our starting point, and that this distance be shorter than the maximum distance. This happens at the step where the list `vecs` is introduced inside `get_mirror_coordinates()`. Lastly, instead of returning a list of tuples, we turn this into a dictionary with the angle (in the form of the arctan) as a key and the length as a value - friendly reminder that both representations of a vector are equivalent :)

That's almost the end, the last problem to take into account is that it's conceivable that a shot fired will hit you before it hits the target. How can we check this? We use the same method `get_mirror_coordinates()` as before, but pass the values for the starting position instead of the target - that will give us all the vectors from which we would should ourselves, given the maximum travel distance.

Once we have both dictionaries, we can check that either
- for an entry in `target_vecs`, no corresponding key exists in `start_vecs` (i.e. by shooting in a given direction, we cannot hit ourselves)
- or if that key also exists in `start_vecs`, its length is longer (meaning that we would hit ourselves only if the target didn't already block the shot)

Full solution below!


In [68]:
import math

def len_vec(v):
    return math.sqrt(pow(v[0],2) + pow(v[1],2))

def diff(v1,v2):
    return (v1[0]-v2[0], v1[1]-v2[1])

def get_mirror_coordinates(dim, start, target, max_dist):
    
    x,y = dim
    tx,ty = target
    
    iterations = (max_dist//min(dim))+1 
    
    # Initialize list for coordinates with values that don't quite follow the standard pattern
    target_range_x = [-tx, tx]
    target_range_y = [-ty, ty]
    
    # Helper function to generate the grid
    func = lambda x,y : [x+y, x-y, y-x, -x-y]
    
    for i in range(1,iterations):
        target_range_x += func(tx,2*i*x)
        target_range_y += func(ty,2*i*y)
    
    # Cartesian product of x and y coordinates after subtracting the starting position
    possible_vecs =  [diff([i,j],start) for i in target_range_x for j in target_range_y]
    # ensure the vectors are not too long
    vecs =  [v for v in possible_vecs if len_vec(v) <= max_dist]
    
    # Create a dictionary with the angle of the vector (key) and its length (value)
    return {math.atan2(v[1], v[0]) : len_vec(v) for v in vecs}

def solution(dim, start, target, max_dist):
    
    # We also need to get all the vectors that would result in hitting ourselves
    start_vecs = get_mirror_coordinates(dim, start, start, max_dist) 
    target_vecs = get_mirror_coordinates(dim, start, target, max_dist)
    
    # check if the shot would hit us before it hits the trainer
    return len([key for key, value in target_vecs.items() if key not in start_vecs or start_vecs[key] > value])
    


In [75]:
get_mirror_coordinates([3,2],[1,1],[2,1],2)

{0.0: 1.0}

In [77]:
solution([340,231], [1,190], [15,200], 400)

6